In [ ]:
class TextureMapGenerator:
    """
    Implementa la generación de mapas de textura según el paper
    "Dynamic Sign Language Recognition Based on Convolutional Neural Networks and Texture Maps".
    
    Genera dos tipos de mapas:
    1. Skeleton Optical Spectra (SOS): Representación del movimiento global de landmarks faciales
    2. Dynamic Images (Rank Pooling): Representación del movimiento local
    """
    
    def __init__(self, width=400, height=400):
        """
        Inicializa el generador de mapas de textura
        
        Args:
            width: Ancho de los mapas de textura generados
            height: Alto de los mapas de textura generados
        """
        self.width = width
        self.height = height
        
        # Parámetros para el modelo de color HSB (ecuación 1 del paper)
        self.hmin = 0
        self.hmax = 360
        self.smin = 0.3
        self.smax = 1.0
        self.bmin = 0.3
        self.bmax = 1.0
        
        # Cache para evitar cálculos repetidos
        self.harmonic_cache = {0: 0}  # Números armónicos para Rank Pooling
    
    def _compute_harmonic_number(self, n):
        """
        Calcula el n-ésimo número armónico H_n = 1 + 1/2 + 1/3 + ... + 1/n
        Usado en la ecuación 5 del paper para Rank Pooling
        """
        if n in self.harmonic_cache:
            return self.harmonic_cache[n]
        
        # Calcular recursivamente usando valores en caché
        if n-1 in self.harmonic_cache:
            h_n = self.harmonic_cache[n-1] + 1/n
        else:
            h_n = sum(1/i for i in range(1, n+1))
        
        self.harmonic_cache[n] = h_n
        return h_n
    
    def _hsb_to_rgb(self, h, s, b):
        """
        Convierte de HSB a RGB (usado para visualización de SOS)
        
        Args:
            h: Hue (0-360)
            s: Saturation (0-1)
            b: Brightness (0-1)
            
        Returns:
            Tupla RGB (0-255, 0-255, 0-255)
        """
        h = h / 360.0
        c = b * s
        x = c * (1 - abs((h * 6) % 2 - 1))
        m = b - c
        
        if h < 1/6:
            r, g, b = c, x, 0
        elif h < 2/6:
            r, g, b = x, c, 0
        elif h < 3/6:
            r, g, b = 0, c, x
        elif h < 4/6:
            r, g, b = 0, x, c
        elif h < 5/6:
            r, g, b = x, 0, c
        else:
            r, g, b = c, 0, x
            
        return (int((r + m) * 255), int((g + m) * 255), int((b + m) * 255))
    
    def generate_sos_maps(self, landmarks_sequence, facial_regions):
        """
        Genera mapas de Skeleton Optical Spectra (SOS) para los landmarks faciales
        Implementa la técnica descrita en la sección II.A.1 del paper
        
        Args:
            landmarks_sequence: Lista de arrays de landmarks faciales (secuencia temporal)
            facial_regions: Diccionario con índices de landmarks para cada región facial
            
        Returns:
            Dict: Mapas SOS para los planos XY, XZ e YZ
        """
        if not landmarks_sequence or len(landmarks_sequence) < 2:
            print("No hay suficientes landmarks para generar mapas SOS")
            return None
        
        # Crear mapas para cada plano
        sos_xy = np.zeros((self.height, self.width, 3), dtype=np.uint8)
        sos_xz = np.zeros((self.height, self.width, 3), dtype=np.uint8)
        sos_yz = np.zeros((self.height, self.width, 3), dtype=np.uint8)
        
        # Filtrar landmarks válidos
        valid_landmarks = [lm for lm in landmarks_sequence if lm is not None and len(lm) > 0]
        
        if len(valid_landmarks) < 2:
            print("No hay suficientes landmarks válidos para generar mapas SOS")
            return None
        
        # Extraer todos los landmarks para normalización
        all_points = []
        for landmarks in valid_landmarks:
            # Convertir landmarks a lista de puntos 2D
            if isinstance(landmarks, np.ndarray) and landmarks.ndim == 3:
                points = landmarks.reshape(-1, 2)
                all_points.extend(points)
        
        if not all_points:
            print("No se pudieron extraer puntos válidos de los landmarks")
            return None
            
        all_points = np.array(all_points)
        
        # Calcular rango para normalización
        min_x, min_y = np.min(all_points, axis=0)
        max_x, max_y = np.max(all_points, axis=0)
        
        range_x = max_x - min_x
        range_y = max_y - min_y
        
        if range_x <= 0 or range_y <= 0:
            print("Rango inválido para normalización de puntos")
            return None
        
        # Calcular velocidades para cada punto facial
        velocities = {}
        for i in range(len(valid_landmarks) - 1):
            try:
                curr_landmarks = valid_landmarks[i]
                next_landmarks = valid_landmarks[i+1]
            
            # Adaptar forma para cálculos
                if isinstance(curr_landmarks, np.ndarray):
                    if curr_landmarks.ndim == 3:
                        curr_reshape = curr_landmarks.reshape(-1, 2)
                    else:
                        curr_reshape = curr_landmarks
                else:
                    continue  # Saltar si no podemos convertir
                
                if isinstance(next_landmarks, np.ndarray):
                    if next_landmarks.ndim == 3:
                        next_reshape = next_landmarks.reshape(-1, 2)
                    else:
                        next_reshape = next_landmarks
                else:
                    continue  # Saltar si no podemos convertir
            
            # Verificar que ambos landmarks tengan al menos un punto
                if len(curr_reshape) == 0 or len(next_reshape) == 0:
                    continue
                
            # Usar solo el número de puntos disponibles en ambos frames
                min_points = min(len(curr_reshape), len(next_reshape))
                
                
                
                for j in range(min_points):
                    if j not in velocities:
                        velocities[j] = []
                    try:
                       if j < len(curr_reshape) and j < len(next_reshape):
                            velocity = np.linalg.norm(next_reshape[j] - curr_reshape[j])
                            velocities[j].append(velocity)
                    except Exception as e:
                        print(f"Error al calcular velocidad para el punto {j}: {str(e)}")
                    # Continuar con el siguiente punto
                        continue
            except Exception as e:
                print(f"Error al procesar par de landmarks {i}: {str(e)}")
                continue

                    
                    
        # Encontrar velocidad máxima para normalización
        max_velocity = 1.0
        for v_list in velocities.values():
            if v_list:
                max_v = np.max(v_list)
                if max_v > max_velocity:
                    max_velocity = max_v
        
        # Definir los 5 grupos de regiones faciales (adaptado del paper)
        # K1 a K5 corresponden a las regiones definidas en el paper
        facial_regions_grouped = {
            'K1': facial_regions.get('indices_ojos', []),             # Equivalente a la región izquierda
            'K2': facial_regions.get('indices_mandibula_menton', []), # Equivalente a la región derecha
            'K3': facial_regions.get('indices_frente_cejas', []),     # Equivalente a brazo izquierdo
            'K4': facial_regions.get('indices_boca', []),             # Equivalente a brazo derecho
            'K5': facial_regions.get('indices_nariz', []) + 
                  facial_regions.get('indices_mejillas_pomulos', []) + 
                  facial_regions.get('indices_arrugas', [])           # Equivalente al cuerpo central
        }
        
        # Función para normalizar coordenadas
        def normalize_coords(x, y, z=0):
            nx = int((x - min_x) / (range_x + 1e-10) * (self.width - 20)) + 10
            ny = int((y - min_y) / (range_y + 1e-10) * (self.height - 20)) + 10
            # Para la coordenada Z, usamos como aproximación la profundidad estimada (podría ser constante)
            nz = int(z * (self.height / 4)) + (self.height // 2)
            
            # Asegurar que las coordenadas estén dentro de los límites
            nx = max(0, min(self.width-1, nx))
            ny = max(0, min(self.height-1, ny))
            nz = max(0, min(self.height-1, nz))
            
            return nx, ny, nz
        
        # Crear un mapeo de cada punto a su región
        point_to_region = {}
        for region_name, indices in facial_regions_grouped.items():
            for idx in indices:
                point_to_region[idx] = region_name
        
        # Determinar a qué región pertenece cada índice en indices_clave
        indices_clave = facial_regions.get('indices_clave', [])
        for idx in indices_clave:
            if idx not in point_to_region:
                # Asignar puntos no clasificados a K5 (región central)
                point_to_region[idx] = 'K5'
        
        # Número total de frames
        n = len(valid_landmarks)
        
        # Dibujar cada punto en los mapas SOS según la ecuación 1 del paper
        for t, landmarks in enumerate(valid_landmarks):
            # Normalizar el tiempo para los cálculos de HSB
            t_norm = t / max(1, n - 1)
            
            # Adaptar forma para cálculos
            if isinstance(landmarks, np.ndarray) and landmarks.ndim == 3:
                landmarks_reshaped = landmarks.reshape(-1, 2)
            else:
                landmarks_reshaped = landmarks
            
            for j, point in enumerate(landmarks_reshaped):
                if j >= len(indices_clave):
                    continue
                    
                idx = indices_clave[j]
                region = point_to_region.get(idx, 'K5')
                
                # Implementar la ecuación 1 del paper para H, S, B
                h = 0
                if region == 'K1':
                    h = (t_norm * (self.hmax - self.hmin)/2) + (self.hmin/2)
                elif region == 'K2':
                    h = (self.hmax/2) - (t_norm * (self.hmax - self.hmin)/2)
                elif region == 'K3':
                    h = self.hmax - (t_norm * (self.hmax - self.hmin)/2)
                elif region == 'K4':
                    h = (self.hmax/2) + (t_norm * (self.hmax - self.hmin)/2)
                elif region == 'K5':
                    h = 0  # Escala de grises para la región central
                
                # Calcular S y B basados en la velocidad
                v_avg = 0
                if j in velocities and velocities[j]:
                    v_idx = min(t, len(velocities[j]) - 1)
                    v_avg = velocities[j][v_idx] / max_velocity
                
                s = self.smin if region == 'K5' else (v_avg * (self.smax - self.smin) + self.smin)
                
                if region == 'K5':
                    b = self.bmax - (t_norm * (self.bmax - self.bmin))
                else:
                    b = (v_avg * (self.bmax - self.bmin) + self.bmin)
                
                # Convertir HSB a RGB
                color = self._hsb_to_rgb(h, s, b)
                
                # Extrar coordenadas X, Y del punto (y generar Z)
                x, y = point[:2]
                z = 0  # No tenemos Z real en landmarks 2D, usamos 0
                
                # Normalizar coordenadas para los tres planos
                nx, ny, nz = normalize_coords(x, y, z)
                
                # Dibujar puntos en los mapas SOS
                cv2.circle(sos_xy, (nx, ny), 2, color, -1)
                cv2.circle(sos_xz, (nx, nz), 2, color, -1)
                cv2.circle(sos_yz, (ny, nz), 2, color, -1)
                
                # Dibujar líneas entre puntos consecutivos para mejorar visualización
                if t > 0 and j < len(valid_landmarks[t-1]):
                    prev_point = valid_landmarks[t-1][j]
                    if isinstance(prev_point, np.ndarray) and prev_point.ndim > 1:
                        prev_point = prev_point.ravel()
                    
                    prev_x, prev_y = prev_point[:2]
                    prev_nx, prev_ny, prev_nz = normalize_coords(prev_x, prev_y, z)
                    
                    # Dibujar líneas solo si los puntos no están muy lejos
                    dist = np.sqrt((nx-prev_nx)**2 + (ny-prev_ny)**2)
                    if dist < 30:  # Umbral para evitar líneas largas
                        cv2.line(sos_xy, (prev_nx, prev_ny), (nx, ny), color, 1)
                        cv2.line(sos_xz, (prev_nx, prev_nz), (nx, nz), color, 1)
                        cv2.line(sos_yz, (prev_ny, prev_nz), (ny, nz), color, 1)
        
        return {
            'DXY': sos_xy,
            'DXZ': sos_xz,
            'DYZ': sos_yz
        }
    
    def generate_dynamic_image(self, frames, normalize=True):
        """
        Genera una imagen dinámica a partir de frames mediante rank pooling
        Implementa la técnica descrita en la sección II.A.2 del paper
        
        Args:
            frames: Lista de frames de vídeo
            normalize: Si True, normaliza la imagen resultante
            
        Returns:
            Imagen dinámica que resume el movimiento
        """
        if not frames or len(frames) < 2:
            print("No hay suficientes frames para generar imagen dinámica")
            return None
        
        # Verifica que todos los frames tengan el mismo tamaño
        height, width = frames[0].shape[:2]
        for frame in frames[1:]:
            if frame.shape[:2] != (height, width):
                print("Los frames deben tener el mismo tamaño")
                return None
        
        # Número total de frames
        T = len(frames)
        
        # Calcular los coeficientes alpha_t según la ecuación 5 del paper
        alphas = np.zeros(T)
        for t in range(1, T+1):  # t comienza desde 1 en el paper
            H_t = self._compute_harmonic_number(t)
            H_t_minus_1 = self._compute_harmonic_number(t-1)
            H_T = self._compute_harmonic_number(T)
            
            # Ecuación 5 del paper
            alphas[t-1] = 2 * (T - t + 1) - (T + 1) * (H_T - H_t_minus_1)
        
        # Normalizar alphas para estabilidad numérica
        alphas = alphas / np.sum(np.abs(alphas))
        
        # Inicializar imagen dinámica
        dynamic_image = np.zeros((height, width, 3), dtype=np.float32)
        
        # Aplicar ecuación 4 del paper: aproximación del rank pooling
        for t in range(T):
            # Convertir frame a float32 para cálculos
            frame = frames[t].astype(np.float32)
            
            # Ponderar frame por su coeficiente alpha_t
            dynamic_image += alphas[t] * frame
        
        if normalize:
            # Normalizar a rango 0-255
            min_val = np.min(dynamic_image)
            max_val = np.max(dynamic_image)
            
            if max_val > min_val:
                dynamic_image = 255 * (dynamic_image - min_val) / (max_val - min_val)
            
        # Convertir a uint8 para visualización
        dynamic_image = np.clip(dynamic_image, 0, 255).astype(np.uint8)
        
        return dynamic_image
    
    def extract_representative_frame(self, frames, landmarks):
        """
        Extrae el frame más representativo de una secuencia, basado en la
        sección II.B del paper (detección de segmento con menor aceleración)
        
        Args:
            frames: Lista de frames
            landmarks: Lista de landmarks correspondientes
            
        Returns:
            Frame representativo con la mejor configuración facial
        """
        if not frames or not landmarks or len(frames) < 3:
            print("No hay suficientes datos para extraer frame representativo")
            return None
        
        # Filtrar landmarks y frames válidos
        valid_data = [(f, l) for f, l in zip(frames, landmarks) if f is not None and l is not None]
        if len(valid_data) < 3:
            print("No hay suficientes datos válidos")
            return None
            
        frames_valid, landmarks_valid = zip(*valid_data)
        
        # 1. Calcular punto central para cada set de landmarks
        center_points = []
        for lm in landmarks_valid:
            if isinstance(lm, np.ndarray):
                if lm.ndim == 3:
                    points = lm.reshape(-1, 2)
                else:
                    points = lm
                center = np.mean(points, axis=0)
                center_points.append(center)
            else:
                # Si no es un array numpy, intentar convertir
                try:
                    points = np.array(lm).reshape(-1, 2)
                    center = np.mean(points, axis=0)
                    center_points.append(center)
                except:
                    print("Error al procesar landmarks, formato no válido")
                    return None
        
        # 2. Calcular distancias entre puntos consecutivos (ecuación 6)
        distances = []
        for i in range(len(center_points) - 1):
            dist = np.linalg.norm(center_points[i+1] - center_points[i])
            distances.append(dist)
        
        # 3. Calcular distancias acumulativas y velocidades (ecuación 7)
        cum_distances = np.cumsum(distances)
        velocities = []
        for i in range(len(cum_distances)):
            velocity = cum_distances[i] / (i + 1)
            velocities.append(velocity)
        
        # 4. Calcular aceleraciones (ecuación 8)
        accelerations = []
        for i in range(len(velocities) - 1):
            acceleration = velocities[i+1] - velocities[i]
            accelerations.append(acceleration)
        
        # 5. Dividir en M=3 segmentos y calcular desviación estándar
        M = 3  # Número de segmentos como en el paper
        if len(accelerations) < M:
            M = len(accelerations)
            
        segments = []
        segment_size = len(accelerations) // M
        
        for i in range(M):
            start_idx = i * segment_size
            end_idx = start_idx + segment_size if i < M-1 else len(accelerations)
            segment = accelerations[start_idx:end_idx]
            sd = np.std(segment)
            segments.append((start_idx, end_idx, sd))
        
        # 6. Seleccionar segmento con menor SD (movimiento más estable)
        min_sd_segment = min(segments, key=lambda x: x[2])
        start_idx, end_idx, _ = min_sd_segment
        
        # Convertir índices de aceleración a índices de frames
        # La aceleración i corresponde al frame i+2
        start_frame_idx = start_idx + 2
        end_frame_idx = min(end_idx + 2, len(frames_valid))
        
        # 7. Extraer frames del segmento y calcular nitidez
        frame_sharpness = []
        for i in range(start_frame_idx, end_frame_idx):
            frame = frames_valid[i]
            
            # Calcular nitidez usando el Laplaciano (medida de energía)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            sharpness = cv2.Laplacian(gray, cv2.CV_64F).var()
            
            frame_sharpness.append((i, sharpness))
        
        if not frame_sharpness:
            # Si no hay frames con nitidez calculada, usar el frame central
            mid_idx = len(frames_valid) // 2
            return frames_valid[mid_idx]
        
        # 8. Seleccionar el frame más nítido
        best_idx, _ = max(frame_sharpness, key=lambda x: x[1])
        
        return frames_valid[best_idx]
    
    def generate_all_texture_maps(self, frames_buffer, landmarks_buffer, facial_regions):
        """
        Genera todos los mapas de textura mencionados en el paper:
        - SOS (Skeleton Optical Spectra): DXY, DXZ, DYZ
        - Dynamic Image (Rank Pooling): DC
        - Frame representativo: HC
        
        Args:
            frames_buffer: Buffer de frames de video
            landmarks_buffer: Buffer de landmarks faciales
            facial_regions: Diccionario con regiones faciales
            
        Returns:
            Dict: Todos los mapas de textura generados
        """
        all_maps = {}
        
        # Filtrar frames y landmarks válidos
        valid_data = []
        for f, l in zip(frames_buffer, landmarks_buffer):
            if f is not None and l is not None:
            # Verificar que landmark tenga datos válidos
                try:
                    if isinstance(l, np.ndarray):
                        if l.size > 0:  # Verificar que no esté vacío
                            valid_data.append((f, l))
                    else:
                        valid_data.append((f, l))
                except:
                # Si hay error al procesar el landmark, omitirlo
                    continue
    
        if len(valid_data) < 3:
            print("No hay suficientes datos válidos para generar mapas de textura")
            return {}
        
        frames_valid, landmarks_valid = zip(*valid_data)
    
        try:
        # 1. Generar mapas SOS
            print("Generando mapas SOS...")
            sos_maps = self.generate_sos_maps(landmarks_valid, facial_regions)
            if sos_maps:
                all_maps.update(sos_maps)
                print("Mapas SOS generados con éxito")
        except Exception as e:
            print(f"Error al generar mapas SOS: {str(e)}")
    
        try:
        # 2. Generar imagen dinámica a color
            print("Generando mapa dinámico DC...")
            dynamic_color = self.generate_dynamic_image(frames_valid)
            if dynamic_color is not None:
                all_maps['DC'] = dynamic_color
                print("Mapa dinámico DC generado con éxito")
        except Exception as e:
            print(f"Error al generar mapa dinámico DC: {str(e)}")
    
        try:
        # 3. Extraer frame representativo
            print("Extrayendo frame representativo HC...")
            representative_frame = self.extract_representative_frame(frames_valid, landmarks_valid)
            if representative_frame is not None:
                all_maps['HC'] = representative_frame
                print("Frame representativo HC extraído con éxito")
        except Exception as e:
            print(f"Error al extraer frame representativo HC: {str(e)}")
    
        return all_maps
    
    
    def visualize_texture_maps(self, texture_maps):
        """
        Visualiza los mapas de textura generados
        
        Args:
            texture_maps: Diccionario con los mapas de textura
            
        Returns:
            Figura de matplotlib con la visualización
        """
        if not texture_maps:
            print("No hay mapas de textura para visualizar")
            return None
        
        num_maps = len(texture_maps)
        fig, axes = plt.subplots(1, num_maps, figsize=(num_maps*5, 5))
        
        # Si solo hay un mapa, axes no es un array
        if num_maps == 1:
            axes = [axes]
        
        # Mostrar cada mapa
        for i, (map_name, texture_map) in enumerate(texture_maps.items()):
            # Convertir de BGR a RGB para matplotlib
            if texture_map.ndim == 3:
                texture_map_rgb = cv2.cvtColor(texture_map, cv2.COLOR_BGR2RGB)
            else:
                texture_map_rgb = texture_map
            
            axes[i].imshow(texture_map_rgb)
            axes[i].set_title(map_name)
            axes[i].axis('off')
        
        plt.tight_layout()
        return fig
    
    def create_enhanced_summary_video(self, texture_maps, frames, output_path, fps=30):
        """
        Crea un video resumen mejorado combinando keyframes con mapas de textura
        Usa los mapas de textura para crear transiciones más suaves y naturales
        
        Args:
            texture_maps: Diccionario con mapas de textura
            frames: Lista de frames para el video
            output_path: Ruta donde guardar el video
            fps: Frames por segundo del video de salida
            
        Returns:
            Ruta al video generado
        """
        if not frames or not texture_maps:
            print("No hay suficientes datos para crear el video")
            return None
        
        # Extraer dimensiones del primer frame
        height, width = frames[0].shape[:2]
        
        # Configurar el escritor de video
        fourcc = cv2.VideoWriter_fourcc(*'MJPG')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
        # Asegurar que todos los mapas de textura tengan el mismo tamaño
        resized_maps = {}
        for name, texture_map in texture_maps.items():
            if texture_map.shape[:2] != (height, width):
                resized_maps[name] = cv2.resize(texture_map, (width, height))
            else:
                resized_maps[name] = texture_map
        
        # Escribir introducción con mapas de textura (2 segundos)
        intro_frames = int(fps * 2)
        for i in range(intro_frames):
            # Crear composición de mapas de textura
            intro_frame = np.zeros((height, width, 3), dtype=np.uint8)
            
            # Mostrar diferentes mapas en secuencia
            map_duration = intro_frames // len(resized_maps)
            map_index = i // map_duration
            map_names = list(resized_maps.keys())
            
            if map_index < len(map_names):
                current_map = resized_maps[map_names[map_index]].copy()
                
                # Añadir texto explicativo
                cv2.putText(current_map, f"Mapa de textura: {map_names[map_index]}", 
                           (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                
                intro_frame = current_map
            
            out.write(intro_frame)
        
        # Escribir secuencia principal con frames y transiciones basadas en mapas
        for i in range(len(frames) - 1):
            # Escribir el frame actual
            out.write(frames[i])
            
            # Crear transición al siguiente frame
            transition_frames = 3  # Número de frames para transición
            for t in range(transition_frames):
                alpha = (t + 1) / (transition_frames + 1)
                
                # Transición básica entre frames
                transition = cv2.addWeighted(frames[i], 1-alpha, frames[i+1], alpha, 0)
                
                # Mejorar la transición con información de mapas de textura
                if 'DXY' in resized_maps:
                    # Usar el mapa DXY para enfatizar movimiento en la transición
                    # Aplicar una ligera mezcla con el mapa de textura
                    transition = cv2.addWeighted(transition, 0.9, 
                                                resized_maps['DXY'], 0.1, 0)
                
                out.write(transition)
        
        # Escribir el último frame
        out.write(frames[-1])
        
        # Cerrar el escritor
        out.release()
        
        print(f"Video resumen mejorado creado en {output_path}")
        return output_path